In [1]:
from pathlib import Path
import polars as pl
from datasets import Dataset
import numpy as np
from autofaiss import build_index
from core_pro.ultilities import make_dir
from sentence_transformers import SentenceTransformer
import sys
sys.path.extend([str(Path.home() / 'PycharmProjects/item_matching')])

from notebooks.benchmark.data_load import load

In [2]:
df, col, path = load()
df.head()

Data Shape: (72110, 6)


id,q_item_id,q_level1_global_be_category,q_item_name,q_link_first_image,q_item_name_clean
u32,i64,str,str,str,str
0,21681018628,"""Men Shoes""","""[OUTLET] Giày Sandal Quai Chéo…","""https://cf.shopee.sg/file/vn-1…","""giày sandal quai chéo nam nữ u…"
1,7669916630,"""Men Shoes""","""Giày Sneaker Nam Đế Cao Su Non…","""https://cf.shopee.sg/file/d771…","""giày sneaker nam đế cao su non…"
2,24932452924,"""Men Shoes""","""Giày thể thao bán chạy*_Adidas…","""https://cf.shopee.sg/file/sg-1…","""giày thể thao bán chạy*_adidas…"
3,26805107273,"""Men Shoes""","""sandal nam DÙ BITIS ❤️DRM03850…","""https://cf.shopee.sg/file/vn-1…","""sandal nam dù bitis ❤️drm03850…"
4,26301825297,"""Men Shoes""","""（Best Quality）Giày Thể Thao Na…","""https://cf.shopee.sg/file/sg-1…","""（best quality）giày thể thao na…"


In [ ]:
path_tmp_array = Path('tmp/array')
path_tmp_ds = Path('tmp/ds')
make_dir(path_tmp_ds)
make_dir(path_tmp_array)

file_embed = path_tmp_array / f'jina_embed.npy'
if not file_embed.exists():
    model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)
    task = 'text-matching'
    embeddings = model.encode(
        df['q_item_name_clean'].to_list(),
        task=task,
        prompt_name=task,
    )
    np.save(file_embed, embeddings)
else:
    embeddings = np.load(file_embed)
print(embeddings.shape)

In [ ]:
df = df.with_columns(pl.Series(values=embeddings, name='embed'))
dataset = Dataset.from_polars(df)
dataset.set_format(type='numpy', columns=['embed'], output_all_columns=True)

In [ ]:
path_index = Path('tmp/index')
build_index(
    str(path_tmp_array),
    index_path=str(path_index / f'ip.index'),
    index_infos_path=str(path_index / f'index.json'),
    save_on_disk=True,
    metric_type='ip',
    verbose=30,
)

In [ ]:
# add index
dataset.load_faiss_index('embed', path_index / f'ip.index')

In [ ]:
score, result = dataset.get_nearest_examples_batch(
    'embed',
    np.asarray(dataset['embed']),
    k=5
)

dict_ = {'score': [list(i) for i in score]}
df_score = pl.DataFrame(dict_)
df_result = pl.DataFrame(result).drop(['embed'])

In [ ]:
df_match = pl.concat([df, df_result, df_score], how='horizontal')
col_explode = [i for i in df_match.columns if 'db' in i] + ['score']
df_match = df_match.explode(col_explode)

In [ ]:
df_match